<a href="https://colab.research.google.com/github/morrisalp/ssh2colab/blob/main/SSH_to_Google_Colab_with_ngrok.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from getpass import getpass

print('Enter ngrok auth token:')
with open('.authtoken', 'w') as f:
  f.write(getpass()) # copy from https://dashboard.ngrok.com/auth/your-authtoken

print('Set SSH password:')
with open('.password', 'w') as f:
  f.write(getpass())

In [ ]:
%%shell
apt-get install openssh-server jq
pushd /usr/local/bin
wget -c -nc https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
unzip -n *.zip
popd
echo "PermitRootLogin yes" >> /etc/ssh/sshd_config
echo "PasswordAuthentication yes" >> /etc/ssh/sshd_config
service ssh start
ngrok authtoken $(cat .authtoken)
echo root:$(cat .password) | chpasswd
nohup ngrok tcp 22 &

In [ ]:
# Run this cell once ngrok has started
tcp_url = ! curl http://localhost:4040/api/tunnels 2>/dev/null | \
            jq -r '.tunnels[0].public_url'
tcp_url = tcp_url[0]
(host, port) = tcp_url.lstrip('tcp://').split(':')
print(f'TCP URL: {tcp_url}')
print(f'SSH command: ssh -p{port} root@{host}')